# Heat Equation in 2D with scikit-fem

We solve the steady-state heat equation (Laplace equation) on a unit square:
$$-\Delta u = 0,\quad (x,y)\in(0,1)^2$$

with Dirichlet boundary conditions:
- $u(x,0) = 1$ (bottom edge: hot)
- $u(x,1) = 0$ (top edge: cold)
- $u(0,y) = 0$ (left edge: cold)
- $u(1,y) = 0$ (right edge: cold)

This models heat conduction in a square plate with a hot bottom edge and cold sides/top.

## Step 1: Import Required Libraries

We import the necessary libraries for 2D finite element computations:
- 2D mesh and triangular elements
- Plotting tools for 2D visualization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skfem as fem
from skfem.helpers import *

## Step 2: Define Mesh and Function Space

Here we create a 2D triangular mesh on the unit square [0,1]×[0,1]:
- `MeshTri.init_tensor` creates a structured triangular mesh
- `ElementTriP1` defines linear (P1) finite elements on triangles
- The mesh is refined to have reasonable resolution

In [ ]:
# Create a structured triangular mesh on unit square
n = 20  # number of subdivisions per direction
mesh = fem.MeshTri.init_tensor(np.linspace(0, 1, n+1), np.linspace(0, 1, n+1))
V = fem.Basis(mesh, fem.ElementTriP1())

print(f"Mesh has {mesh.p.shape[1]} nodes and {mesh.t.shape[1]} triangles")

# Visualize the mesh (coarse version for clarity)
if n <= 10:
    plt.figure(figsize=(6,6))
    mesh.plot()
    plt.title('Triangular Mesh')
    plt.axis('equal')
    plt.show()

## Step 3: Define Weak Form (Variational Formulation)

The weak form of the Laplace equation $-\Delta u = 0$ is: Find $u$ such that
$$\int_{\Omega} \nabla u \cdot \nabla v \, dx dy = 0 \quad \forall v$$

We define:
- **Bilinear form**: $a(u,v) = \int \nabla u \cdot \nabla v \, dx dy$
- **Linear form**: $L(v) = 0$ (no source term)

In 2D, $\nabla u \cdot \nabla v = \frac{\partial u}{\partial x}\frac{\partial v}{\partial x} + \frac{\partial u}{\partial y}\frac{\partial v}{\partial y}$

In [ ]:
# Define bilinear form: ∫ ∇u · ∇v dx dy
@fem.BilinearForm
def a(u, v, _):
    return dot(grad(u), grad(v))  # dot computes ∇u · ∇v in any dimension

# Define linear form: ∫ 0 * v dx dy (no source term)
@fem.LinearForm  
def L(v, _):
    return 0.0 * v

## Step 4: Assembly

We assemble the stiffness matrix and load vector. Since we have no source term, the load vector will be zero (before applying boundary conditions).

In [ ]:
# Assemble stiffness matrix and load vector
A = a.assemble(V)
b = L.assemble(V)

print(f"System size: {A.shape[0]} x {A.shape[1]}")
print(f"Load vector norm: {np.linalg.norm(b):.2e} (should be ~0)")

## Step 5: Apply Boundary Conditions and Solve

We identify boundary nodes and apply Dirichlet conditions:
- Bottom edge (y=0): $u = 1$
- All other edges: $u = 0$

The boundary condition application modifies both the matrix and right-hand side.

In [ ]:
# Get node coordinates
x, y = m.p[0], m.p[1]

# Find boundary nodes
tol = 1e-12
bottom = np.where(np.abs(y) < tol)[0]          # y = 0 (hot)
top = np.where(np.abs(y - 1.0) < tol)[0]       # y = 1 (cold)
left = np.where(np.abs(x) < tol)[0]            # x = 0 (cold)
right = np.where(np.abs(x - 1.0) < tol)[0]     # x = 1 (cold)

print(f"Boundary nodes: bottom={len(bottom)}, top={len(top)}, left={len(left)}, right={len(right)}")

# Apply boundary conditions
A_dense = A.toarray()

# Bottom edge: u = 1
for i in bottom:
    A_dense[i, :] = 0
    A_dense[i, i] = 1
    b[i] = 1.0

# Other edges: u = 0
for boundary_nodes in [top, left, right]:
    for i in boundary_nodes:
        A_dense[i, :] = 0
        A_dense[i, i] = 1
        b[i] = 0.0

# Solve the system
u = np.linalg.solve(A_dense, b)
print(f"Solution computed with {len(u)} degrees of freedom")
print(f"Solution range: [{u.min():.3f}, {u.max():.3f}]")

## Step 6: Visualization

We visualize the temperature distribution using:
- Contour plot showing isotherms (lines of constant temperature)
- Color map showing temperature variation
- 3D surface plot for better understanding of the solution

The solution shows heat diffusion from the hot bottom edge, with temperature decreasing toward the cold boundaries.

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Contour plot
ax1 = axes[0]
levels = np.linspace(0, 1, 11)
cs = ax1.tricontourf(x, y, u, levels=levels, cmap='hot')
ax1.tricontour(x, y, u, levels=levels, colors='black', alpha=0.3, linewidths=0.5)
plt.colorbar(cs, ax=ax1, label='Temperature')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Temperature Distribution')
ax1.set_aspect('equal')

# 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_trisurf(x, y, u, cmap='hot', alpha=0.8)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('Temperature')
ax2.set_title('3D Temperature Surface')
plt.colorbar(surf, ax=ax2, shrink=0.5)

plt.tight_layout()
plt.show()

# Temperature profile along centerline (x=0.5)
plt.figure(figsize=(8, 4))
center_idx = np.where(np.abs(x - 0.5) < 0.02)[0]  # nodes near x=0.5
center_y = y[center_idx]
center_u = u[center_idx]
sort_idx = np.argsort(center_y)

plt.plot(center_y[sort_idx], center_u[sort_idx], 'o-', linewidth=2, markersize=6)
plt.xlabel('y (height)')
plt.ylabel('Temperature')
plt.title('Temperature Profile along Centerline (x=0.5)')
plt.grid(True, alpha=0.3)
plt.show()

## Analysis and Insights

The solution shows several important features:

1. **Boundary conditions**: Temperature is 1 at the bottom and 0 on other edges
2. **Heat diffusion**: Temperature decreases smoothly from bottom to top
3. **Corner effects**: Sharp gradients near corners where boundary conditions change
4. **Symmetry**: The solution is symmetric about the vertical centerline

This is a classic example of steady-state heat conduction governed by Laplace's equation.

### Experiments
- Change the boundary conditions (e.g., make left/right edges hot)
- Add a heat source by modifying the linear form
- Try different mesh resolutions and observe convergence
- Implement mixed boundary conditions (Neumann on some edges)